In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Adım 1: Yapılandırılmış Çıktılar için Pydantic Modellerini Tanımlayın

Bu modeller, ajanların döndüreceği **şemayı** tanımlar. Pydantic ile `response_format` kullanmak şu avantajları sağlar:
- ✅ Tür güvenli veri çıkarımı
- ✅ Otomatik doğrulama
- ✅ Serbest metin yanıtlarından kaynaklanan ayrıştırma hataları yok
- ✅ Alanlara dayalı kolay koşullu yönlendirme


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Adım 2: Otel Rezervasyon Aracını Oluşturun

Bu araç, **availability_agent**'in odaların müsait olup olmadığını kontrol etmek için çağıracağı araçtır. `@ai_function` dekoratörünü kullanarak:
- Bir Python fonksiyonunu AI tarafından çağrılabilir bir araca dönüştürürüz
- LLM için otomatik olarak JSON şeması oluştururuz
- Parametre doğrulamasını yönetiriz
- Ajanlar tarafından otomatik çağrılmayı mümkün kılarız

Bu demo için:
- **Stockholm, Seattle, Tokyo, Londra, Amsterdam** → Odalar mevcut ✅
- **Diğer tüm şehirler** → Odalar mevcut değil ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Adım 3: Yönlendirme için Koşul Fonksiyonlarını Tanımlayın

Bu fonksiyonlar, ajanın yanıtını inceler ve iş akışında hangi yolun izleneceğine karar verir.

**Temel Kalıp:**
1. Mesajın `AgentExecutorResponse` olup olmadığını kontrol edin
2. Yapılandırılmış çıktıyı (Pydantic modeli) analiz edin
3. Yönlendirmeyi kontrol etmek için `True` veya `False` döndürün

İş akışı, hangi yürütücünün bir sonraki adımda çağrılacağını belirlemek için bu koşulları **kenarlarda** değerlendirecektir.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Adım 4: Özel Görüntüleme Executor'ı Oluşturma

Executor'lar, dönüşümler veya yan etkiler gerçekleştiren iş akışı bileşenleridir. Sonucu görüntüleyen özel bir executor oluşturmak için `@executor` dekoratörünü kullanıyoruz.

**Temel Kavramlar:**
- `@executor(id="...")` - Bir işlevi iş akışı executor'ı olarak kaydeder
- `WorkflowContext[Never, str]` - Girdi/çıktı için tür ipuçları
- `ctx.yield_output(...)` - Nihai iş akışı sonucunu verir


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Adım 5: Ortam Değişkenlerini Yükle

LLM istemcisini yapılandırın. Bu örnek şu hizmetlerle çalışır:
- **GitHub Modelleri** (GitHub token ile ücretsiz katman)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Adım 6: Yapılandırılmış Çıktılarla AI Ajanları Oluşturma

**Üç özel ajan** oluşturuyoruz, her biri bir `AgentExecutor` içinde sarılmış:

1. **availability_agent** - Araç kullanarak otel müsaitliğini kontrol eder
2. **alternative_agent** - Alternatif şehirler önerir (odalar yoksa)
3. **booking_agent** - Rezervasyonu teşvik eder (odalar varsa)

**Ana Özellikler:**
- `tools=[hotel_booking]` - Ajana aracı sağlar
- `response_format=PydanticModel` - Yapılandırılmış JSON çıktısını zorlar
- `AgentExecutor(..., id="...")` - İş akışı kullanımı için ajanı sarar


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Adım 7: Koşullu Kenarlarla İş Akışını Oluşturma

Şimdi, koşullu yönlendirme ile grafiği oluşturmak için `WorkflowBuilder` kullanıyoruz:

**İş Akışı Yapısı:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Ana Metotlar:**
- `.set_start_executor(...)` - Giriş noktasını belirler
- `.add_edge(from, to, condition=...)` - Koşullu kenar ekler
- `.build()` - İş akışını sonlandırır


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Adım 8: Test Durumu 1'i Çalıştır - Uygunluk OLMAYAN Şehir (Paris)

Simülasyonumuzda odası olmayan Paris'teki otelleri talep ederek **uygunluk yok** yolunu test edelim.


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Adım 9: Test Durumu 2'yi Çalıştır - Müsaitlik Olan Şehir (Stockholm)

Şimdi, Stockholm'deki otelleri (simülasyonumuzda odaları olan) talep ederek **müsaitlik** yolunu test edelim.


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Önemli Noktalar ve Sonraki Adımlar

### ✅ Öğrendikleriniz:

1. **WorkflowBuilder Deseni**
   - Giriş noktasını tanımlamak için `.set_start_executor()` kullanın
   - Koşullu yönlendirme için `.add_edge(from, to, condition=...)` kullanın
   - İş akışını tamamlamak için `.build()` çağrısı yapın

2. **Koşullu Yönlendirme**
   - Koşul fonksiyonları `AgentExecutorResponse`'ı inceler
   - Yapılandırılmış çıktıları analiz ederek yönlendirme kararları alın
   - Bir kenarı etkinleştirmek için `True`, atlamak için `False` döndürün

3. **Araç Entegrasyonu**
   - Python fonksiyonlarını AI araçlarına dönüştürmek için `@ai_function` kullanın
   - Ajanlar ihtiyaç duyduğunda araçları otomatik olarak çağırır
   - Araçlar, ajanların analiz edebileceği JSON döndürür

4. **Yapılandırılmış Çıktılar**
   - Tip güvenli veri çıkarımı için Pydantic modellerini kullanın
   - Ajanlar oluştururken `response_format=MyModel` ayarını yapın
   - Yanıtları `Model.model_validate_json()` ile analiz edin

5. **Özel Yürütücüler**
   - İş akışı bileşenleri oluşturmak için `@executor(id="...")` kullanın
   - Yürütücüler veri dönüştürebilir veya yan etkiler gerçekleştirebilir
   - İş akışı sonuçları üretmek için `ctx.yield_output()` kullanın

### 🚀 Gerçek Dünya Uygulamaları:

- **Seyahat Rezervasyonu**: Müsaitlik kontrolü, alternatif öneriler, seçenek karşılaştırma
- **Müşteri Hizmetleri**: Sorun türüne, duygu durumuna, önceliğe göre yönlendirme
- **E-ticaret**: Stok kontrolü, alternatif öneriler, sipariş işleme
- **İçerik Moderasyonu**: Zehirlilik puanlarına, kullanıcı işaretlerine göre yönlendirme
- **Onay İş Akışları**: Miktara, kullanıcı rolüne, risk seviyesine göre yönlendirme
- **Çok Aşamalı İşleme**: Veri kalitesine, tamlığa göre yönlendirme

### 📚 Sonraki Adımlar:

- Daha karmaşık koşullar ekleyin (birden fazla kriter)
- İş akışı durum yönetimi ile döngüler uygulayın
- Yeniden kullanılabilir bileşenler için alt iş akışları ekleyin
- Gerçek API'lerle entegre edin (otel rezervasyonu, stok sistemleri)
- Hata yönetimi ve yedek yollar ekleyin
- Dahili görselleştirme araçlarıyla iş akışlarını görselleştirin



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluğu sağlamak için çaba göstersek de, otomatik çevirilerin hata veya yanlışlık içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul edilmez.
